In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from pose_dataset import PoseData

In [2]:
INPUT_SIZE = 33 * 3
HIDDEN_SIZE = 64
NUM_LAYERS = 2 # number of RNNs to stack
NUM_CLASSES = 9 # number of categories

LEARNING_RATE = 0.1

In [3]:
TIME_DIM = 0
BATCH_DIM = 1
COORD_DIM = 2

In [ ]:
class PoseScoringModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(PoseScoringModel, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        h0 = torch.zeros(self.num_layers, X.size(BATCH_DIM), self.hidden_size).to(device)

        out, _ = self.rnn(X, h0)
        # out: batch x time x hidden
        out = out[:, -1, :]
        # out: batch x hidden
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

In [ ]:
model = PoseScoringModel(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES)

In [8]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
NUM_WORKERS = 2

In [ ]:
data = PoseData("PoseData")
data.shuffle()

train_data = data[:len(data) * .9]
test_data = data[len(data) * .9:]

train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)
# train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
# test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

In [ ]:
import os
from sklearn.model_selection import KFold

EPOCHS = 10  # Number of epochs for training
model.train()  # Set the model to training mode

# Create a directory to save snapshots if it doesn't exist
os.makedirs("snapshots", exist_ok=True)
# Number of splits for k-fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)

# Convert dataset to a tensor for splitting
data_tensor = torch.tensor(data)

for fold, (train_idx, val_idx) in enumerate(kf.split(data_tensor)):
    print(f"Fold {fold + 1}/{k_folds}")
    
    # Split data into training and validation sets
    train_subset = data_tensor[train_idx]
    val_subset = data_tensor[val_idx]
    
    train_dataloader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_dataloader = DataLoader(val_subset, batch_size=32, shuffle=False)
    
    for epoch in range(EPOCHS):
        model.train()
        for i, (inputs, targets) in enumerate(train_dataloader):
            # Forward pass
            outputs = model(inputs)
            loss_value = loss(outputs, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()

            if i % 10 == 0:
                print(f"Fold [{fold + 1}/{k_folds}], Epoch [{epoch + 1}/{EPOCHS}], "
                      f"Step [{i + 1}/{len(train_dataloader)}], Loss: {loss_value.item():.4f}")

        # Evaluate on validation set
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in val_dataloader:
                outputs = model(inputs)
                val_loss += loss(outputs, targets).item()
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
        
        val_accuracy = 100 * correct / total
        print(f"Fold [{fold + 1}/{k_folds}], Epoch [{epoch + 1}/{EPOCHS}], "
              f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")
        # Save a snapshot of the model at each epoch
        torch.save(model.state_dict(), f"snapshots/model_fold{fold + 1}_epoch{epoch + 1}.pth")

Epoch [1/10], Loss: 2.2096
Epoch [2/10], Loss: 2.1685
Epoch [3/10], Loss: 2.2148
Epoch [4/10], Loss: 2.3354
Epoch [5/10], Loss: 2.1534
Epoch [6/10], Loss: 2.3413
Epoch [7/10], Loss: 2.3165
Epoch [8/10], Loss: 2.3018
Epoch [9/10], Loss: 2.3047
Epoch [10/10], Loss: 2.3223


In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
eval_loss = 0

with torch.no_grad():  # Disable gradient computation for evaluation
    for inputs, targets in test_dataloader:
        # Forward pass
        outputs = model(inputs)
        eval_loss += loss(outputs, targets).item()

        # Get predictions and calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

accuracy = 100 * correct / total
print(f"Evaluation Loss: {eval_loss:.4f}, Accuracy: {accuracy:.2f}%")